In [1]:
import datetime, time
import simpy

import shapely.geometry
from simplekml import Kml, Style

import pandas as pd
import openclsim.core as core
import openclsim.model as model

import datetime

import numpy as np

from plot import vessel_planning

In [2]:
simulation_start = datetime.datetime.now()

my_env = simpy.Environment(initial_time=time.mktime(simulation_start.timetuple()))
my_env.epoch = time.mktime(simulation_start.timetuple())

registry = {}
keep_resources = {}

In [3]:
Site = type(
    "Site",
    (
        core.Identifiable,
        core.Log,
        core.Locatable,
        core.HasContainer,
        core.HasResource,
    ),
    {},
)


TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,
        core.Log,
        core.ContainerDependentMovable,
        core.Processor,
        core.HasResource,
        core.LoadingFunction,
        core.UnloadingFunction,
    ),
    {},
)

In [4]:
location_from_site = shapely.geometry.Point(5.1, 52)
location_to_site = shapely.geometry.Point(5, 52.1)
location_to_site_2 = shapely.geometry.Point(5, 52.2)

data_from_site = {
    "env": my_env,
    "name": "Winlocatie",
    "geometry": location_from_site,
    "capacity": 100,
    "level": 100,
}

data_to_site = {
    "env": my_env,
    "name": "Dumplocatie",
    "geometry": location_to_site,
    "capacity": 55,
    "level": 0,
}


from_site = Site(**data_from_site)
to_site = Site(**data_to_site)


init
level: 100
completed init
init
level: 0
completed init


In [5]:
data_cutter = {
    "env": my_env,
    "name": "Cutter_1",
    "geometry": location_from_site,
    "capacity": 5,
    "compute_v": lambda x: 10,
    "loading_rate": 3600/5,
    "unloading_rate": 3600/5
}


data_barge_1 = {
    "env": my_env,
    "name": "Barge_1",
    "geometry": location_to_site,
    "capacity": 5,
    "compute_v": lambda x: 10,
    "loading_rate": 3600/5,
    "unloading_rate": 3600/5
}


data_barge_2 = {
    "env": my_env,
    "name": "Barge_2",
    "geometry": location_to_site,
    "capacity": 5,
    "compute_v": lambda x: 10,
    "loading_rate": 3600/5,
    "unloading_rate": 3600/5
}

cutter = TransportProcessingResource(**data_cutter)
barge_1 = TransportProcessingResource(**data_barge_1)
barge_2 = TransportProcessingResource(**data_barge_2)

init
level: 0
completed init
init
level: 0
completed init
init
level: 0
completed init


In [6]:
requested_resources = {}

single_run = [
    model.MoveActivity(**{
        "env": my_env,
        "name": "sailing empty_1",
        "registry": registry,
        "mover": barge_1,
        "destination": from_site,
        "postpone_start": True,
    }),
    model.ShiftAmountActivity(**{
        "env": my_env,
        "name": "Transfer MP_1",
        "registry": registry,
        "processor": cutter,
        "origin": from_site,
        "destination": barge_1,
        "amount": 5,
        "duration": 3600,
        "postpone_start": True,
#         "requested_resources":requested_resources
    }),
    model.MoveActivity(**{
        "env": my_env,
        "name": "sailing filled_1",
        "registry": registry,
        "mover": barge_1,
        "destination": to_site,
        "postpone_start": True,
    }),
    model.ShiftAmountActivity(**{
        "env": my_env,
        "name": "Transfer TP_1",
        "registry": registry,
        "processor": barge_1,
        "origin": barge_1,
        "destination": to_site,
        "amount": 5,
        "duration": 3600,
        "postpone_start": True,
#         "requested_resources":requested_resources
    })
]


sequential_activity_data = {
    "env": my_env,
    "name": "Single run process_1",
    "registry": registry,
    "sub_processes": single_run,
    "postpone_start": True,
}
activity = model.SequentialActivity(**sequential_activity_data)

while_activity = model.WhileActivity(**{
    "env": my_env,
    "name": "while_1",
    "registry": registry,
    "sub_process": activity,
    "condition_event": [{"type":"container", "concept": to_site, "state":"full"}],
    "postpone_start": False,
})

while Activity keep_resources []
get_full_event : default
start get_available
start event instance None


In [7]:
single_run = [
    model.MoveActivity(**{
        "env": my_env,
        "name": "sailing empty_2",
        "registry": registry,
        "mover": barge_2,
        "destination": from_site,
        "postpone_start": True,
    }),
    model.ShiftAmountActivity(**{
        "env": my_env,
        "name": "Transfer MP_2",
        "registry": registry,
        "processor": cutter,
        "origin": from_site,
        "destination": barge_2,
        "amount": 5,
        "duration": 3600,
        "postpone_start": True,
    }),
    model.MoveActivity(**{
        "env": my_env,
        "name": "sailing filled_2",
        "registry": registry,
        "mover": barge_2,
        "destination": to_site,
        "postpone_start": True,
    }),
    model.ShiftAmountActivity(**{
        "env": my_env,
        "name": "Transfer TP_2",
        "registry": registry,
        "processor": barge_2,
        "origin": barge_2,
        "destination": to_site,
        "amount": 5,
        "duration": 3600,
        "postpone_start": True,
    })
]


activity = model.SequentialActivity(** {
    "env": my_env,
    "name": "Single run process_2",
    "registry": registry,
    "sub_processes": single_run,
    "postpone_start": True,
#     "keep_resources"
})

while_activity = model.WhileActivity(**{
    "name": "while_2",
    "env": my_env,
    "registry": registry,
    "sub_process": activity,
    "condition_event": [{"type":"container", "concept": to_site, "state":"full"}],
    "postpone_start": False,
})

while Activity keep_resources []
get_full_event : default
start get_available
start event instance None


In [8]:
my_env.run()

conditional 
start event instance None
keep_resources []
start event instance None
Mover_move before mover resource request
conditional 
start event instance None
keep_resources []
start event instance None
Mover_move before mover resource request
put_callback - id_ default
{}
put_callback - id_ default
{'default': {55: <Event() object at 0x1a622e98d88>}}
amount :55
put_callback - id_ default
{}
put_callback - id_ default
{}
put_callback - id_ default
{}
Mover_move after mover resource request
Mover_move after mover resource request
Mover_move after move
Activity end(sailing empty_1)
Mover_move after move
Activity end(sailing empty_2)
keep_resources []
SHift amount Activity keep_resources []
start event instance None
start : {}
{'origin.333fc574-abdc-11ea-b1c1-2079183a5c9a': 100, 'destination.334320db-abdc-11ea-aff1-2079183a5c9a': 5}
keep_resources []
SHift amount Activity keep_resources []
start event instance None
start : {<simpy.resources.resource.Resource object at 0x000001A622DA23

Mover_move after mover resource request
Mover_move after move
Activity end(sailing empty_2)
Mover_move after move
Activity end(sailing empty_1)
keep_resources []
SHift amount Activity keep_resources []
start event instance None
start : {}
{'origin.333fc574-abdc-11ea-b1c1-2079183a5c9a': 70, 'destination.334347d4-abdc-11ea-bdf9-2079183a5c9a': 5}
keep_resources []
SHift amount Activity keep_resources []
start event instance None
start : {<simpy.resources.resource.Resource object at 0x000001A622DA2408>: <Request() object at 0x1a622da2748>}
{'origin.333fc574-abdc-11ea-b1c1-2079183a5c9a': 70, 'destination.334320db-abdc-11ea-aff1-2079183a5c9a': 5}
destination request : {<simpy.resources.resource.Resource object at 0x000001A622DA2408>: <Request() object at 0x1a622da2748>, <simpy.resources.resource.Resource object at 0x000001A622DA2388>: <Request() object at 0x1a61da22a88>}
shift amount process keep_resources []
start get_available
destination request : {<simpy.resources.resource.Resource objec

destination request : {<simpy.resources.resource.Resource object at 0x000001A620837608>: <Request() object at 0x1a622d9dfc8>}
shift amount process keep_resources []
start get_available
destination request : {<simpy.resources.resource.Resource object at 0x000001A620837608>: <Request() object at 0x1a622d9dfc8>}
shift amount process keep_resources []
start get_available
processor request : {<simpy.resources.resource.Resource object at 0x000001A620837608>: <Request() object at 0x1a622d9dfc8>, <simpy.resources.resource.Resource object at 0x000001A622DA2388>: <Request() object at 0x1a622b18dc8>, <simpy.resources.resource.Resource object at 0x000001A622DA2408>: <Request() object at 0x1a622dc3048>}
site request : {<simpy.resources.resource.Resource object at 0x000001A620837608>: <Request() object at 0x1a622d9dfc8>, <simpy.resources.resource.Resource object at 0x000001A622DA2388>: <Request() object at 0x1a622b18dc8>, <simpy.resources.resource.Resource object at 0x000001A622DA2408>: <Request() o

Mover_move after mover resource request
Mover_move after move
Activity end(sailing filled_1)
Mover_move after move
Activity end(sailing filled_2)
keep_resources []
SHift amount Activity keep_resources []
start event instance None
start : {}
{'origin.334320db-abdc-11ea-aff1-2079183a5c9a': 5, 'destination.3340139a-abdc-11ea-b92c-2079183a5c9a': 15}
keep_resources []
SHift amount Activity keep_resources []
start event instance None
start : {<simpy.resources.resource.Resource object at 0x000001A620837608>: <Request() object at 0x1a622dc1b88>}
{'origin.334347d4-abdc-11ea-bdf9-2079183a5c9a': 5, 'destination.3340139a-abdc-11ea-b92c-2079183a5c9a': 15}
destination request : {<simpy.resources.resource.Resource object at 0x000001A620837608>: <Request() object at 0x1a622dc1b88>}
shift amount process keep_resources []
start get_available
destination request : {<simpy.resources.resource.Resource object at 0x000001A620837608>: <Request() object at 0x1a622dc1b88>}
shift amount process keep_resources []

In [9]:
# !! BUGG !!
#  why we need reservations:
# !! BUGG !!

print(f"Level of the to_site: {to_site.container.get_level()}, Capacity of the to_site {to_site.container.get_capacity()}")

Level of the to_site: 60, Capacity of the to_site 55


In [10]:
objects = [cutter, barge_1, barge_2]
activities = []
for obj in objects:
    print(set(obj.log["Message"]))
    activities.extend(set(obj.log["Message"]))

activities = list(set(activities))

            
C = np.linspace(0,255, len(activities))
colors = {}

for i in range(len(activities)):
    colors[i] = f'rgb({int(C[i]/2)},{int(C[len(C) - 1 - i]/1.5)},{int(C[i])})'

vessel_planning(objects, activities, colors)

{'transfer default to Barge_2', 'transfer default to Barge_1'}
{'sailing filled', 'sailing empty', 'transfer default to Dumplocatie', 'transfer default to Barge_1'}
{'sailing filled', 'sailing empty', 'transfer default to Barge_2', 'transfer default to Dumplocatie'}


In [11]:
cutter.resource.capacity

1